# Tune Model for QA

In [ ]:
!nvidia-smi

Sat Nov 18 15:24:38 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA A100-SXM...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   32C    P0    44W / 400W |      0MiB / 40960MiB |      0%      Default |
|                               |                      |             Disabled |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

# Finetuning Source code
The fine-tuning code developed for version 0.1.0-beta is fully compatible with version 1.0.0-beta. This is because both versions share the same underlying architecture, Llama v2. Therefore, the same codebase can be effectively utilized for fine-tuning the 1.0.0-beta version.

In [ ]:
!git clone https://github.com/OpenThaiGPT/openthaigpt-finetune.git

Cloning into 'openthaigpt-finetune'...
remote: Enumerating objects: 60, done.
remote: Counting objects: 100% (60/60), done.
remote: Compressing objects: 100% (49/49), done.
remote: Total 60 (delta 22), reused 36 (delta 9), pack-reused 0
Receiving objects: 100% (60/60), 20.77 MiB | 12.40 MiB/s, done.
Resolving deltas: 100% (22/22), done.


In [ ]:
%cd openthaigpt-finetune

/content/openthaigpt-finetune


### Install Requirements

In [ ]:
! pip install -r requirements.txt

  Cloning https://github.com/huggingface/peft.git to /tmp/pip-req-build-qvdryget
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/peft.git /tmp/pip-req-build-qvdryget
  Resolved https://github.com/huggingface/peft.git to commit f1ecfa6ae6eba599ae89decbf47b339d8c9d39a3
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.4/261.4 kB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.6/92.6 MB 19.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 91.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 55.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.3/88.3 kB 14.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 83.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━

# Dataset Preparation for Fine-Tuning

When preparing a dataset for fine-tuning, consider the following guidelines:

- **Format Preference**: The dataset should be in JSON Lines (JSONL) format. This format maintains each data entry on a separate line, enhancing readability and processing efficiency.

- **Data Structure**: Each entry in the dataset should contain three fields:
  - `instruction`: Describes the task or context for the model.
  - `input`: Provides the actual input that the model will process.
  - `output`: Contains the expected output or response from the model.

- **Dataset Size**: Aim for a minimum of 10,000 pairs (input-output sets). This size is suggested to provide enough data for effective learning, though the focus should remain on data quality.

- **Quality Over Quantity**: Prioritize the quality of data. High-quality data leads to better model performance, even if the overall quantity is lower.

- **Data Variance**: Ensure diversity in the dataset. Varied data helps in training a model that is robust and performs well across different types of inputs and scenarios.


## Convert Dataset to JSONL format

In [ ]:
from datasets import load_dataset

dataset = load_dataset("parinzee/claq-qa-thai-dataset")
dataset

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/16980 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['id', 'question', 'context', 'answers', 'source', 'th_aug', 'th_fasttext_aug', 'th_llm_gec_aug', 'th_llm_paraphrase_aug', 'th_ltw2v_aug', 'th_qcpg_0.2_aug', 'th_qcpg_0.2_llm_gec_aug', 'th_qcpg_0.5_aug', 'th_qcpg_0.5_llm_gec_aug', 'th_qcpg_0.8_aug', 'th_qcpg_0.8_llm_gec_aug', 'th_thai2fit_aug', 'th_thai2trans_aug', 'th_wordnet_aug', 'en_aug', 'en_llm_gec_aug', 'en_llm_paraphrase_aug', 'en_qcpg_0.2_aug', 'en_qcpg_0.2_llm_gec_aug', 'en_qcpg_0.5_aug', 'en_qcpg_0.5_llm_gec_aug', 'en_qcpg_0.8_aug', 'en_qcpg_0.8_llm_gec_aug', 'dis_aug', 'dis_fasttext_aug', 'dis_llm_gec_aug', 'dis_llm_paraphrase_aug', 'dis_ltw2v_aug', 'dis_qcpg_0.2_aug', 'dis_qcpg_0.2_llm_gec_aug', 'dis_qcpg_0.5_aug', 'dis_qcpg_0.5_llm_gec_aug', 'dis_qcpg_0.8_aug', 'dis_qcpg_0.8_llm_gec_aug', 'dis_thai2fit_aug', 'dis_thai2trans_aug', 'dis_wordnet_aug'],
        num_rows: 16980
    })
})

In [ ]:
df = dataset["train"].to_pandas()
df

,id,question,context,answers,source,th_aug,th_fasttext_aug,th_llm_gec_aug,th_llm_paraphrase_aug,th_ltw2v_aug,...,dis_ltw2v_aug,dis_qcpg_0.2_aug,dis_qcpg_0.2_llm_gec_aug,dis_qcpg_0.5_aug,dis_qcpg_0.5_llm_gec_aug,dis_qcpg_0.8_aug,dis_qcpg_0.8_llm_gec_aug,dis_thai2fit_aug,dis_thai2trans_aug,dis_wordnet_aug
0,82c589fda5d3007be968f303e04a9ac58953299a64a5be...,พัทธ์ธีรา ศรุติพงศ์โภคิน เกิดวันที่เท่าไร?,พัทธ์ธีรา ศรุติพงศ์โภคิน (เกิด 3 ธันวาคม พ.ศ. ...,"{'text': ['3 ธันวาคม พ.ศ. 2533'], 'answer_star...",iapp,Phatthira Sarutpong Phokin วันเดือนปีเกิดคืออะไร?,พัณณ์ธีภ ศรุติวัฒน์2โภคิน เกิดวันที่ไหร่?,วันเดือนปีเกิดของ Phatthira Sarutpong Phokin ค...,Phatthira Sarutpong Phokin คุณช่วยจัดเตรียมวัน...,พัทธ์ธีรา ศรุติพงษ์โภคิน ทำให้เกิดวันอาทิตย์เท...,...,0.379822,0.457964,0.442132,0.442132,0.664607,0.373971,0.391374,0.213284,0.000000,9.606189e-02
1,6483e513603bd34b6f23a5fbe9e1c1db194ce4f4365a69...,พัทธ์ธีรา ศรุติพงศ์โภคิน มีฃื่อเล่นว่าอะไร?,พัทธ์ธีรา ศรุติพงศ์โภคิน (เกิด 3 ธันวาคม พ.ศ. ...,"{'text': ['อร'], 'answer_start': [68], 'answer...",iapp,Phatthira Sarutpong Phokin คุณกำลังเล่นอะไรอยู่?,พัทธ์ธีรา ศิษฎพงศ์อรัช dcccxxxii ในPSPเล่นว่าอ...,คุณกำลังเล่นอะไร Phatthira Sarutpong Phokin?,Phatthira Sarutpong Phokin คุณกำลังเล่นเกมอะไร...,พัทธ์ธีรา ศรุติพงษ์โภคิน มีฃื่อเล่นว่าอะไร?,...,0.028428,0.512295,0.604564,0.552465,0.590820,0.583206,0.645776,0.232190,0.000000,0.000000e+00
2,09fc57198420c10d8cd67154a154cca37fdcde53ec894a...,พัทธ์ธีรา ศรุติพงศ์โภคิน ทำอาชีพอะไร?,พัทธ์ธีรา ศรุติพงศ์โภคิน (เกิด 3 ธันวาคม พ.ศ. ...,"{'text': ['นักแสดงหญิงชาวไทย'], 'answer_start'...",iapp,อาชีพของ Phatthira Teerathiyapong Phokin คืออะไร?,รพนธ์ธีคาต ศรุติวรรณ2โภคิน ทำอาชีพอะไร4?,อาชีพของ Phatthira Teerathiyapong Phokin คืออะไร?,อาชีพของ Phatthira Teerathiyapong Phokin คืออะไร?,พัทธ์ธีรา ศรุติพงษ์โภคิน ทำอาชีพอะไร?,...,0.017806,0.408607,0.408607,0.346143,0.346143,0.328919,0.328919,0.150410,0.000000,0.000000e+00
3,7b2f336d3dbc29c6fb1edf7f4988f968ed0a1272fe8aff...,พัทธ์ธีรา ศรุติพงศ์โภคิน จบการศึกษาจากประเทศอะไร?,พัทธ์ธีรา ศรุติพงศ์โภคิน (เกิด 3 ธันวาคม พ.ศ. ...,"{'text': ['ประเทศนิวซีแลนด์'], 'answer_start':...",iapp,Phatthira Sarutpong Phokin จบการศึกษาจากประเทศ...,พัทธ์ธีรา ตอัตถพงศ์ฯกุล1 จบการศึกษาจากภ.ต่างอะ...,Phatthira Sarutpong Phokin จบการศึกษาจากประเทศใด?,Phatthira Sarutpong Phokin จากประเทศใดจากการศึ...,พัทธ์ธีรา ศรุติพงษ์โภคิน จบการศึกษาจากในประเทศ...,...,0.044393,0.600420,0.717404,0.606477,0.717404,0.433692,0.523367,0.155829,0.009204,1.820072e-01
4,50c66c986cdf19b60f36da1b89137a6ceb4f11aa9ed0e9...,บิดาของคลีโอพัตราเป็นใคร?,คลีโอพัตราที่ 7 ฟิโลพาเธอร์ (กรีก: Κλεοπάτρα θ...,"{'text': ['ทอเลมีที่ 12 ออเลติส'], 'answer_sta...",iapp,พ่อของคลีโอพัตราคือใคร?,บิดาของมเนสโอพัตตกฺขหลายๆใคร?,พ่อของคลีโอพัตราคือใคร?,พ่อผู้ให้กำเนิดของคลีโอพัตราคือใคร?,มารดาของคลีโอพัตราเป็นใคร?,...,0.180941,0.238710,0.388979,0.151006,0.443043,0.097685,0.097685,0.150196,0.156331,8.621460e-02
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16975,e780cc4812dd88f21ec7f0152905b15588d1a6faab66d2...,หนังสือการ์ตูนชานะ นักรบเนตรอัคคี มีกี่เล่ม?,นิยาย ชานะ นักรบเนตรอัคคี แต่งโดย ยาชิจิโร ทาค...,"{'text': ['22', '26'], 'answer_start': [267, 9...",tydiqa,หนังสือการ์ตูนนักรบดับเพลิงกี่คน?,หนังสือDIYการ์ตูนPokemonชานะ อุดมการณ์ถ.ทิพยเน...,นักรบดับเพลิงมีกี่คนในหนังสือการ์ตูน?,ในหนังสือการ์ตูนมีนักรบดับเพลิงกี่คน?,หนังสือการ์ตูนชาน่ะ นักรบเนตรอัคคี มีกี่เล่ม?,...,0.038205,0.459041,0.345795,0.330124,0.345795,0.363513,0.284700,0.063899,0.012518,1.700147e-01
16976,dd424979403f3909773a832c4d83818e564fe7bfeb4f38...,ไบโอช็อก อินฟินิต เปิดตัวครั้งแรกเมื่อไหร่?,ไบโอช็อก อินฟินิต (English: BioShock Infinite)...,"{'text': ['26 มีนาคม พ.ศ. 2556', '26 มีนาคม พ....",tydiqa,BioShock Infin เปิดตัวครั้งแรกเมื่อใด?,ไบโอมีช๊อค อร์เนชยมีฟิตี้07ต เปิดตัวครั้งแรกเม...,BioShock Infinite เปิดตัวครั้งแรกเมื่อใด?,การเปิดตัวครั้งแรกของ BioShock Infinite คือเมื...,ไบโอช็อค อินฟินิต เผยโฉมเป็นครั้งแรกเมื่อไร?,...,0.168088,0.609325,0.713147,0.475034,0.686884,0.482104,0.602809,0.097

In [ ]:
import ast
df["answers"] = df["answers"].apply(ast.literal_eval)
df

,id,question,context,answers,source,th_aug,th_fasttext_aug,th_llm_gec_aug,th_llm_paraphrase_aug,th_ltw2v_aug,...,dis_ltw2v_aug,dis_qcpg_0.2_aug,dis_qcpg_0.2_llm_gec_aug,dis_qcpg_0.5_aug,dis_qcpg_0.5_llm_gec_aug,dis_qcpg_0.8_aug,dis_qcpg_0.8_llm_gec_aug,dis_thai2fit_aug,dis_thai2trans_aug,dis_wordnet_aug
0,82c589fda5d3007be968f303e04a9ac58953299a64a5be...,พัทธ์ธีรา ศรุติพงศ์โภคิน เกิดวันที่เท่าไร?,พัทธ์ธีรา ศรุติพงศ์โภคิน (เกิด 3 ธันวาคม พ.ศ. ...,"{'text': ['3 ธันวาคม พ.ศ. 2533'], 'answer_star...",iapp,Phatthira Sarutpong Phokin วันเดือนปีเกิดคืออะไร?,พัณณ์ธีภ ศรุติวัฒน์2โภคิน เกิดวันที่ไหร่?,วันเดือนปีเกิดของ Phatthira Sarutpong Phokin ค...,Phatthira Sarutpong Phokin คุณช่วยจัดเตรียมวัน...,พัทธ์ธีรา ศรุติพงษ์โภคิน ทำให้เกิดวันอาทิตย์เท...,...,0.379822,0.457964,0.442132,0.442132,0.664607,0.373971,0.391374,0.213284,0.000000,9.606189e-02
1,6483e513603bd34b6f23a5fbe9e1c1db194ce4f4365a69...,พัทธ์ธีรา ศรุติพงศ์โภคิน มีฃื่อเล่นว่าอะไร?,พัทธ์ธีรา ศรุติพงศ์โภคิน (เกิด 3 ธันวาคม พ.ศ. ...,"{'text': ['อร'], 'answer_start': [68], 'answer...",iapp,Phatthira Sarutpong Phokin คุณกำลังเล่นอะไรอยู่?,พัทธ์ธีรา ศิษฎพงศ์อรัช dcccxxxii ในPSPเล่นว่าอ...,คุณกำลังเล่นอะไร Phatthira Sarutpong Phokin?,Phatthira Sarutpong Phokin คุณกำลังเล่นเกมอะไร...,พัทธ์ธีรา ศรุติพงษ์โภคิน มีฃื่อเล่นว่าอะไร?,...,0.028428,0.512295,0.604564,0.552465,0.590820,0.583206,0.645776,0.232190,0.000000,0.000000e+00
2,09fc57198420c10d8cd67154a154cca37fdcde53ec894a...,พัทธ์ธีรา ศรุติพงศ์โภคิน ทำอาชีพอะไร?,พัทธ์ธีรา ศรุติพงศ์โภคิน (เกิด 3 ธันวาคม พ.ศ. ...,"{'text': ['นักแสดงหญิงชาวไทย'], 'answer_start'...",iapp,อาชีพของ Phatthira Teerathiyapong Phokin คืออะไร?,รพนธ์ธีคาต ศรุติวรรณ2โภคิน ทำอาชีพอะไร4?,อาชีพของ Phatthira Teerathiyapong Phokin คืออะไร?,อาชีพของ Phatthira Teerathiyapong Phokin คืออะไร?,พัทธ์ธีรา ศรุติพงษ์โภคิน ทำอาชีพอะไร?,...,0.017806,0.408607,0.408607,0.346143,0.346143,0.328919,0.328919,0.150410,0.000000,0.000000e+00
3,7b2f336d3dbc29c6fb1edf7f4988f968ed0a1272fe8aff...,พัทธ์ธีรา ศรุติพงศ์โภคิน จบการศึกษาจากประเทศอะไร?,พัทธ์ธีรา ศรุติพงศ์โภคิน (เกิด 3 ธันวาคม พ.ศ. ...,"{'text': ['ประเทศนิวซีแลนด์'], 'answer_start':...",iapp,Phatthira Sarutpong Phokin จบการศึกษาจากประเทศ...,พัทธ์ธีรา ตอัตถพงศ์ฯกุล1 จบการศึกษาจากภ.ต่างอะ...,Phatthira Sarutpong Phokin จบการศึกษาจากประเทศใด?,Phatthira Sarutpong Phokin จากประเทศใดจากการศึ...,พัทธ์ธีรา ศรุติพงษ์โภคิน จบการศึกษาจากในประเทศ...,...,0.044393,0.600420,0.717404,0.606477,0.717404,0.433692,0.523367,0.155829,0.009204,1.820072e-01
4,50c66c986cdf19b60f36da1b89137a6ceb4f11aa9ed0e9...,บิดาของคลีโอพัตราเป็นใคร?,คลีโอพัตราที่ 7 ฟิโลพาเธอร์ (กรีก: Κλεοπάτρα θ...,"{'text': ['ทอเลมีที่ 12 ออเลติส'], 'answer_sta...",iapp,พ่อของคลีโอพัตราคือใคร?,บิดาของมเนสโอพัตตกฺขหลายๆใคร?,พ่อของคลีโอพัตราคือใคร?,พ่อผู้ให้กำเนิดของคลีโอพัตราคือใคร?,มารดาของคลีโอพัตราเป็นใคร?,...,0.180941,0.238710,0.388979,0.151006,0.443043,0.097685,0.097685,0.150196,0.156331,8.621460e-02
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16975,e780cc4812dd88f21ec7f0152905b15588d1a6faab66d2...,หนังสือการ์ตูนชานะ นักรบเนตรอัคคี มีกี่เล่ม?,นิยาย ชานะ นักรบเนตรอัคคี แต่งโดย ยาชิจิโร ทาค...,"{'text': ['22', '26'], 'answer_start': [267, 9...",tydiqa,หนังสือการ์ตูนนักรบดับเพลิงกี่คน?,หนังสือDIYการ์ตูนPokemonชานะ อุดมการณ์ถ.ทิพยเน...,นักรบดับเพลิงมีกี่คนในหนังสือการ์ตูน?,ในหนังสือการ์ตูนมีนักรบดับเพลิงกี่คน?,หนังสือการ์ตูนชาน่ะ นักรบเนตรอัคคี มีกี่เล่ม?,...,0.038205,0.459041,0.345795,0.330124,0.345795,0.363513,0.284700,0.063899,0.012518,1.700147e-01
16976,dd424979403f3909773a832c4d83818e564fe7bfeb4f38...,ไบโอช็อก อินฟินิต เปิดตัวครั้งแรกเมื่อไหร่?,ไบโอช็อก อินฟินิต (English: BioShock Infinite)...,"{'text': ['26 มีนาคม พ.ศ. 2556', '26 มีนาคม พ....",tydiqa,BioShock Infin เปิดตัวครั้งแรกเมื่อใด?,ไบโอมีช๊อค อร์เนชยมีฟิตี้07ต เปิดตัวครั้งแรกเม...,BioShock Infinite เปิดตัวครั้งแรกเมื่อใด?,การเปิดตัวครั้งแรกของ BioShock Infinite คือเมื...,ไบโอช็อค อินฟินิต เผยโฉมเป็นครั้งแรกเมื่อไร?,...,0.168088,0.609325,0.713147,0.475034,0.686884,0.482104,0.602809,0.097

In [ ]:
filtered_df = df[(df["source"] != "tydiqa") & (df["source"] != "xquad")].reset_index(drop=True)

In [ ]:
all_questions = filtered_df["question"].to_list() + filtered_df["th_llm_gec_aug"].to_list()
len(all_questions)

22554

In [ ]:
all_contexts = filtered_df["context"].to_list() * 2
len(all_contexts)

22554

In [ ]:
all_ans = list(filtered_df["answers"].apply(lambda x: x["text"][0])) * 2
len(all_ans)

22554

In [ ]:
import pandas as pd
final_df = pd.DataFrame({
    "input": all_contexts,
    "output": all_ans,
    "instruction": all_questions,
})

In [ ]:
final_df

,input,output,instruction
0,พัทธ์ธีรา ศรุติพงศ์โภคิน (เกิด 3 ธันวาคม พ.ศ. ...,3 ธันวาคม พ.ศ. 2533,พัทธ์ธีรา ศรุติพงศ์โภคิน เกิดวันที่เท่าไร?
1,พัทธ์ธีรา ศรุติพงศ์โภคิน (เกิด 3 ธันวาคม พ.ศ. ...,อร,พัทธ์ธีรา ศรุติพงศ์โภคิน มีฃื่อเล่นว่าอะไร?
2,พัทธ์ธีรา ศรุติพงศ์โภคิน (เกิด 3 ธันวาคม พ.ศ. ...,นักแสดงหญิงชาวไทย,พัทธ์ธีรา ศรุติพงศ์โภคิน ทำอาชีพอะไร?
3,พัทธ์ธีรา ศรุติพงศ์โภคิน (เกิด 3 ธันวาคม พ.ศ. ...,ประเทศนิวซีแลนด์,พัทธ์ธีรา ศรุติพงศ์โภคิน จบการศึกษาจากประเทศอะไร?
4,คลีโอพัตราที่ 7 ฟิโลพาเธอร์ (กรีก: Κλεοπάτρα θ...,ทอเลมีที่ 12 ออเลติส,บิดาของคลีโอพัตราเป็นใคร?
...,...,...,...
22549,เรวิญานันท์ ทาเกิด เรวิญานันท์ ทาเกิด หรือ เบญ...,จังหวัดลำพูน,บ้านเกิดหรือบ้านเกิดของ Revyanan คืออะไร?
22550,กูดฟีลลิง (เพลง) กูดฟิลลิง เป็นเพลงของนักร้องแ...,ไวลด์วันส์,อัลบั้ม Gud Philing คืออะไร?
22551,ปภัสรา เตชะไพบูลย์ ดร.ปภัสรา เตชะไพบูลย์ มีชื่...,นายพรเทพ เตชะไพบูลย์,ใครคือ Kob Papasara และนักการเมืองคนไหนที่เธอแ...
22552,ปีเตอร์ ดรักเกอร์ ปีเตอร์ เฟอร์ดินานด์ ดรักเกอ...,ทนายความ,พ่อของ Peter Dergers ทำอะไร?


In [ ]:
! mkdir data

mkdir: cannot create directory ‘data’: File exists


In [ ]:
final_df.to_json("data/claq-qa.jsonl",orient='records', lines=True)

In [ ]:
final_df.shape

(22554, 3)

In [ ]:
!ls

alpaca_data_cleaned_archive.json  docker-compose.yml		   generate.py	     templates
alpaca_data_gpt4.json		  Dockerfile			   LICENSE	     utils
alpaca_data.json		  export_hf_checkpoint.py	   pyproject.toml
data				  export_state_dict_checkpoint.py  README.md
DATA_LICENSE			  finetune.py			   requirements.txt


# LoRA Method for Supervised Finetuning
Low-Rank Adaptation: LoRA involves introducing low-rank matrices to modify existing weights in a pre-trained model. Instead of training all parameters of the model, which can be prohibitively large, LoRA focuses on training these smaller, added matrices.

# Fine-Tuning Command Explanation

- **Base Model**: Specifies the model to fine-tune, here 'openthaigpt/openthaigpt-1.0.0-beta-7b-chat-ckpt-hf'.
- **Data Path**: Sets the directory of training data, 'data/'.
- **Output Directory**: The output directory for fine-tuned model, './kumpun-output'.
- **Batch Size**: Number of samples per batch, set to 1.
- **Micro Batch Size**: Size of micro batches, also set to 1.
- **Number of Epochs**: Total training epochs, set to 100.
- **Learning Rate**: The learning rate for training, 1e-3.
- **Cutoff Length**: Maximum sequence length, 256.
- **Validation Set Size**: Size of the validation set, 10.
- **LoRA Parameters**: Settings for Low-Rank Adaptation - 'r' is 4, 'alpha' is 16, 'dropout' is 0.05, target modules are '[q_proj,v_proj]'.
- **Train on Inputs**: Indicates training on input sequences.
- **Group by Length**: Enables grouping samples by their lengths for efficiency.

With Epoch = 3, It supposed to wait for 8 minutes.

In [ ]:
! python finetune.py \
    --base_model 'ChanonUtupon/openthaigpt-test-llama-2-7B-th-3.5k' \
    --data_path 'data/' \
    --output_dir './claq-qa-v2-output' \
    --batch_size 64 \
    --micro_batch_size 8 \
    --num_epochs 3 \
    --learning_rate 1.5e-4 \
    --cutoff_len 256 \
    --val_set_size 4000 \
    --lora_r 8 \
    --lora_alpha 16 \
    --lora_dropout 0.05 \
    --lora_target_modules '[q_proj,v_proj]' \
    --train_on_inputs \
    --group_by_length

Training Llama v2 model with params:
base_model: ChanonUtupon/openthaigpt-test-llama-2-7B-th-3.5k
data_path: data/
output_dir: ./claq-qa-v2-output
batch_size: 64
micro_batch_size: 8
num_epochs: 3
learning_rate: 0.00015
cutoff_len: 256
val_set_size: 4000
lora_r: 8
lora_alpha: 16
lora_dropout: 0.05
lora_target_modules: ['q_proj', 'v_proj']
train_on_inputs: True
add_eos_token: False
group_by_length: True
wandb_project: 
wandb_run_name: 
wandb_watch: 
wandb_log_model: 
resume_from_checkpoint: False
prompt template: llama_v2

Loading checkpoint shards: 100% 66/66 [00:29<00:00,  2.25it/s]
/usr/local/lib/python3.10/dist-packages/peft/utils/other.py:136: FutureWarning: prepare_model_for_int8_training is deprecated and will be removed in a future version. Use prepare_model_for_kbit_training instead.
  warnings.warn(
trainable params: 4,194,304 || all params: 6,943,756,288 || trainable%: 0.060403963302232765
Map: 100% 18554/18554 [01:16<00:00, 242.39 examples/s]
Map: 100% 4000/4000 [00:16<00:00,

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!cp -r claq-qa-v2-output ../drive/MyDrive/openthaiclaq-v2-output

# Generate Command Summary

- **8-bit Loading**: Enables loading the model in 8-bit format (`--load_8bit`).
- **Base Model**: Specifies the base model, 'openthaigpt/openthaigpt-1.0.0-beta-7b-chat-ckpt-hf'.
- **LoRA Weights**: Path to LoRA weights, './kumpun-output'.
- **Share Gradio**: Indicates sharing via Gradio, set to 'true'.

In [ ]:
! pip install gradio==3.44.4

In [ ]:
! python generate.py \
    --base_model 'ChanonUtupon/openthaigpt-test-llama-2-7B-th-3.5k' \
    --lora_weights 'claq-qa-v2-output/checkpoint-600' \
    --share_gradio 'true'

base_model: ChanonUtupon/openthaigpt-test-llama-2-7B-th-3.5k
lora_weights: claq-qa-v2-output/checkpoint-600
Loading checkpoint shards: 100% 66/66 [00:31<00:00,  2.11it/s]
/content/openthaigpt-finetune/generate.py:192: GradioDeprecationWarning: Usage of gradio.inputs is deprecated, and will not be supported in the future, please import your component from gradio.components
  gr.inputs.Textbox(
/content/openthaigpt-finetune/generate.py:192: GradioDeprecationWarning: `optional` parameter is deprecated, and it has no effect
  gr.inputs.Textbox(
/content/openthaigpt-finetune/generate.py:192: GradioDeprecationWarning: `numeric` parameter is deprecated, and it has no effect
  gr.inputs.Textbox(
Running on local URL:  http://0.0.0.0:7860
Running on public URL: https://966fd233fd99767a97.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
/usr/local/lib/python3.10/dis

# Uploading a Model to Hugging Face

To upload a model to Hugging Face, follow these streamlined steps:

1. **Hugging Face Login**: Start by logging into your Hugging Face account. This step is crucial for authentication and accessing the right privileges.

2. **Create a Model Repository**: On Hugging Face, create a new repository specifically for your model. This repository will store all relevant files and data.

3. **Clone Repository in Colab**: Use Git commands within Google Colab to clone the newly created repository. This action allows you to work directly with the repository files in a Colab environment.

4. **Add Model Files**: Insert your model files into the cloned repository. These files include the trained model, configuration files, and any additional necessary documents.

5. **Commit and Push**: Commit the changes to the Git repository and then push them to Hugging Face. This process uploads your local changes to the online repository.

6. **Verify on Hugging Face**: Finally, check your model on Hugging Face's platform. Ensure that all files are correctly uploaded and the model version is accessible.


In [ ]:
! git config --global credential.helper store

In [ ]:
! huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Token: 
Add token as git credential? (Y/n) y
Token is valid (permission: write).
Your token has been saved in your configured git credential helpers (store).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [ ]:
!git lfs install

Updated git hooks.
Git LFS initialized.


## Creating a New Model Repository on Hugging Face

Follow these steps to create a new model repository on Hugging Face:

1. **Log in to Hugging Face**: Ensure you're logged into your Hugging Face account. If you don’t have an account, sign up for one.

2. **Go to the Models Section**: On the Hugging Face homepage, navigate to the 'Models' section. This is where all the models are listed and managed.

3. **Create New Model**: Look for the option to create a new model. Usually, this is a button labeled 'New' or 'Create New Model'.

4. **Fill in Details**: In the creation form, enter the necessary details:
   - Model name: Choose a unique and descriptive name for your model.
   - Visibility: Decide whether your model will be public or private.
   - License: Select an appropriate license for your model.
   - Tags and Readme: Add relevant tags for easy discovery and a Readme file to describe your model, its use cases, and any other important information.

5. **Create Repository**: After filling in all details, click the button to create the repository. This action will set up a new model repository under your Hugging Face account.

6. **Access Your Repository**: Once created, you can access your repository to upload model files, manage versions, and share your model with the community.


In [ ]:
!git clone https://huggingface.co/parinzee/villiam-qa-100-beta-7b

Cloning into 'villiam-qa-100-beta-7b'...
remote: Enumerating objects: 4, done.
remote: Total 4 (delta 0), reused 0 (delta 0), pack-reused 4
Unpacking objects: 100% (4/4), 490 bytes | 490.00 KiB/s, done.


In [ ]:
! cp /content/openthaigpt-finetune/claq-qa-v2-output/checkpoint-600/adapter* villiam-qa-100-beta-7b

In [ ]:
%cd villiam-qa-100-beta-7b

/content/openthaigpt-finetune/villiam-qa-100-beta-7b


In [ ]:
! git config --global user.email "parinzee@protonmail.com"
! git config --global user.name "Parinthapat Pengpun"
! git add --all
! git commit -am "Initial Version"

[main 4118854] Initial Version
 2 files changed, 26 insertions(+)
 create mode 100644 adapter_config.json
 create mode 100644 adapter_model.safetensors


In [ ]:
! git push origin

Uploading LFS objects: 100% (1/1), 17 MB | 3.0 MB/s, done.
Enumerating objects: 5, done.
Counting objects: 100% (5/5), done.
Delta compression using up to 12 threads
Compressing objects: 100% (4/4), done.
Writing objects: 100% (4/4), 783 bytes | 783.00 KiB/s, done.
Total 4 (delta 0), reused 0 (delta 0), pack-reused 0
To https://huggingface.co/parinzee/villiam-qa-100-beta-7b
   312bb56..4118854  main -> main


### Visit your huggingface model's page
For example, https://huggingface.co/kobkrit/openthaigpt-100-beta-7b-kumpun/

In [ ]:
! git add --all
! git commit -am "Updated Readme"

[main 3853fc0] Updated Readme
 1 file changed, 171 insertions(+), 3 deletions(-)
 rewrite README.md (100%)


In [ ]:
!git push origin

Enumerating objects: 5, done.
Counting objects: 100% (5/5), done.
Delta compression using up to 12 threads
Compressing objects: 100% (3/3), done.
Writing objects: 100% (3/3), 2.73 KiB | 2.73 MiB/s, done.
Total 3 (delta 1), reused 0 (delta 0), pack-reused 0
remote: -------------------------------------------------------------------------
remote: Your push was accepted, but with warnings: 
remote: - Warning: empty or missing yaml metadata in repo card
remote: help: https://huggingface.co/docs/hub/model-cards#model-card-metadata
remote: -------------------------------------------------------------------------
remote: -------------------------------------------------------------------------
remote: Please find the documentation at:
remote: https://huggingface.co/docs/hub/model-cards#model-card-metadata
remote: 
remote: -------------------------------------------------------------------------
To https://huggingface.co/parinzee/villiam-qa-100-beta-7b
   4118854..3853fc0  main -> main
